## Outline

Today we will show the Process of classificating Time-series starting with simple TS data using a sliding window approach
* First we show the preprocessing of a timeseries given as a continouus vector to a usable Dataloader
* Next we will show how to train a ConvNet using PyTorch on a Time-series classification Task
* We will show how to optimize parameters like Kernelsize, and number of kernels (i.e. number of features) etc. using grid search

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np

from torch.utils.data import Dataset, ConcatDataset, DataLoader, random_split

# Time Series Classification Using CNN

The Data consist of three types of time series
* Step function
* Sine Wave
* Sawtooth Wave

To make it more difficult for the classification, a strong noise is added to the functions

In [ ]:
def get_ts_steps(n, m, noise):
    """Sample a simple random two-class time series"""
    s = np.random.random((n)) * noise - noise / 2
    k = int(len(s) / m)
    for i in range(m):
        if i % 2 == 0:
            continue
        s[k * i:k * (i + 1)] += +1
    return s

def get_ts_wave(n, m, noise):
    """Sample a simple random two-class time series"""
    x = np.sin(np.linspace(-np.pi, m*np.pi, n))
    s = np.random.random((n)) * noise - noise / 2
    return x+s

def get_Sawtooth_wave(n, m, noise):
    """Sample a simple random two-class time series"""
    s = np.random.random((n)) * noise - noise / 2 - 1
    k = int(len(s) / m)
    lin_func = lambda x: 2/k * x
    lin = lin_func(np.arange(k))
    for i in range(m):
        if i % 2 == 0:
            continue
        s[k * i:k * (i + 1)] += 1*lin
    return s

>First we generate long Timeseries with n = 50000 datapoints each

In [ ]:
n = 10000 
m = 500
noise = .1

X_steps = get_ts_steps(n, m, noise)
X_wave = get_ts_wave(n, m, noise)
X_sawtooth = get_Sawtooth_wave(n, m, noise)

**1)** Plot first 200 points of each Timeseries

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

####################
# Your Code Here   #
####################

**2)** Create a Dataloader for Training

>A dataloader is necessary for loading a consistent representation of values in the input space for the training, evaluation and test process. 
The idea is to train a model without dependecy of the length of the whole dataset. 
In this case we have 3 synthetic time series with 50000 data points. Fitting all the data at once into a model would lead to high computational requirements. 

>We will use a sliding window approach to split the data into smaller pieces and the length of the segments extracted here should be considered as a hyperparameter value. 

[Tutorial](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) gives an overview over the process. The Dataset class must implement three functions: 
` __init__`
` __len__`
` __getitem__`

for our case we need the additional function:
`split_sequence` to split the long time series into segments

In [ ]:
class TimeSeriesData(Dataset):
    def __init__(self, X, y, sw):
        
        self.X = X # Time Series data
        self.y = y # class
        self.sw = sw # window size of the sliding window
        
    def __len__(self):
        
        # The __len__ function returns the number of samples in our dataset
        # hint: you need to take into account the window size s
        
        ####################
    # Your Code Here   #
    ####################
        
    def split_sequence(self, X, y, index):
        
        # The split_sequence class returns the time series vector from the given index 
        # up to the given index plus the window size 
        # Additionaly it gives the realted class
        
        ####################
    # Your Code Here   #
    ####################
    
    def __getitem__(self, idx):  
    
        # The __getitem__ function returns a sample from the dataset at the given index idx
        # It should return it as a torch.Tensor class
        
        ####################
    # Your Code Here   #
    ####################

**3)** Creating Dataloader for training, testing and validation

* Create Datasets for each Time series seperatly
* Concat the datasets using [ConcatDataset](https://pytorch.org/docs/stable/data.html)
* Split the DataSet using [random_split](https://pytorch.org/docs/stable/data.html)
* Create Dataloader using [DataLoader](https://pytorch.org/docs/stable/data.html)

In [ ]:
sw = 50

steps_DS = TimeSeriesData(X_steps, y=np.ones(len(X_steps))*0, sw = sw) # Dataset for class 0, step_function
wave_DS = TimeSeriesData(X_wave, y=np.ones(len(X_steps))*1, sw = sw)   # Dataset for class 1, wave_function
sawtooth_DS = TimeSeriesData(X_sawtooth, y=np.ones(len(X_steps))*2, sw = sw) # Dataset for class 2, sawtooth-function

# concat the datasets
####################
# Your Code Here   #
####################

# split the dataset in trainset, valset and testset with a 70%, 15%, 15% split
####################
# Your Code Here   #
####################

# Create Dataloader for training, testing an valitation
batch_size = 13
####################
# Your Code Here   #
####################


In [ ]:
x_, y_ = next(iter(loader_train))

In [ ]:
y_.shape

**4)** Implement a CNN architecture

>Use one 1D Convolutional Layer with Relu activation and max pooling (kernel = 2). After the convolutional Layer one Linear layer need to be applied. Lastly the classification should be realized by a Sofmax Function <br>
<br>
hint: you need to take into account the different output dimensions, according to convolution and pooling outup <br>
<br>
Convolutional output ([link](https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html)): <br>
<br>
$$L_{out} = \left[\frac{L_{in} + 2\cdot padding - dilation\cdot(kernel\_size - 1) - 1}{stride} + 1\right]$$
<br>
<br>
padding = 0 <br>
stride = 1 <br>
dilation = 1 <br>
<br>
pooling output ([link](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool1d.html))

In [ ]:
class CNN(nn.Module):
    def __init__(self, input_size, n_feature, kernel_size, output_size):
        super(CNN, self).__init__()
        ####################
    # Your Code Here   #
    ####################
        
    def forward(self, x):
        ####################
    # Your Code Here   #
    ####################
    

**5)** Implement a training and testing function

In [ ]:
def train(epoch, model, optimizer, loader_train):
    model.train()
    for batch_idx, (data, target) in enumerate(loader_train):
        # Here you need to implement a training process, which uses the given optimizer 
        # claculates the loss and backprobs through the network
        # hint: take a look at previous implementations in other excersices
        ####################
    # Your Code Here   #
    ####################
    

**6)** Implement a grid search optimization for the Numer of kernels ([`out_channels`](https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html)) and the kernel size

>The next step is to implement a short grid search for the Number of features and the kernel size, use an algorithm like in excercise 10, optimization. You can use the previously defined training and testing functions to make you code look more clear


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
n_feature = [1, 2, 3, 4, 5, 6]
kernel_size = [1, 3, 5, 7, 9]
epochs = 3
results_acc_list = []
results_loss_list = []

for feature in n_feature:
    results_acc = []
    results_loss = []
    for kernel in kernel_size:

        ####################
    # Your Code Here   #
    ####################
        
    results_acc_list.append(results_acc)
    results_loss_list.append(results_loss)

**7)** Visualize the optimization results

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
#ax.view_init(30, 60)
X, Y = np.meshgrid(n_feature, kernel_size)
ax.plot_surface(X, Y, np.vstack(results_loss_list).transpose(), cmap="plasma")
ax.set_xlabel('n_feature', fontsize=15, rotation=60)
ax.set_ylabel('kernel_size', fontsize=15, rotation=60)
plt.title('Loss')

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
#ax.view_init(30, 225)
X, Y = np.meshgrid(n_feature, kernel_size)
ax.plot_surface(X, Y, np.vstack(results_acc_list).transpose(), cmap="plasma")
ax.set_xlabel('n_feature', fontsize=15, rotation=60)
ax.set_ylabel('kernel_size', fontsize=15, rotation=60)
plt.title('Accuracy')

**8)** Final Results: Use the test Set to evaluate the Model with the best Hyperparameters

In [ ]:
####################
# Your Code Here   #
####################

**9)** What other hyperparameter can be optimized? 

>Discussion